As per usual the code in the repo for AIMA is, well, broken. So in answering question 4.4 from chapter 4 I fixed it. Note that I am using the search_2 module, which is part of the AIMA repository. See https://github.com/hmp-anthony. In this post I am comparing steepest-ascent hill-climbing ,random-restart hill-climbing and simulated annealing. To keep things fair I am using the same `find_neighbor` method for all three algorithms. This method just selects a random contiguous sub-array, reverses it and puts it back together. The problems that these algorithms will be working on are the 8puzzle problem and the 8queens problem. These are well-known problems in AI, google them! :)

In [31]:
from search_2 import *
from utils import *
import time

class EightPuzzle(Problem):

    def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal

    def two_opt(self, state):
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state))
        right = random.randint(0, len(neighbour_state))
        if left > right:
            left, right = right, left
        neighbour_list = list(neighbour_state)
        x = neighbour_list[left: right + 1]
        neighbour_list[left: right + 1] = x[::-1]
        neighbour_state = tuple(neighbour_list)
        return neighbour_state
        
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)

    def value(self, state):
        """ value of path cost given negative for the given state """
        return -1 * hamming_distance(state, self.goal)

def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))



class EightQueensProblem(Problem):

    def __init__(self):
        self.initial = tuple(range(8))
        Problem.__init__(self, self.initial)

    def two_opt(self, state):
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state))
        right = random.randint(0, len(neighbour_state))
        if left > right:
            left, right = right, left
        neighbour_list = list(neighbour_state)
        x = neighbour_list[left: right + 1]
        neighbour_list[left: right + 1] = x[::-1]
        neighbour_state = tuple(neighbour_list)
        return neighbour_state
        

    def result(self, state, row):
        """Place the next queen at the given row."""
        col = state.index(-1)
        new = list(state[:])
        new[col] = row
        return tuple(new)
    
    def conflicted(self, state, row, col):
        """Would placing a queen at (row, col) conflict with anything?"""
        return any(self.conflict(row, col, state[c], c)
                   for c in range(col))

    def conflict(self, row1, col1, row2, col2):
        """Would putting two queens in (row1, col1) and (row2, col2) conflict?"""
        return (row1 == row2 or  # same row
                col1 == col2 or  # same column
                row1 - col1 == row2 - col2 or  # same \ diagonal
                row1 + col1 == row2 + col2)   # same / diagonal

    def goal_test(self, state):
        """Check if all columns filled, no conflicts."""
        if state[-1] == -1:
            return False
        return not any(self.conflicted(state, state[col], col)
                       for col in range(len(state)))

    def value(self, state):
        """Return number of conflicting queens for a given node"""
        num_conflicts = 0
        for (r1, c1) in enumerate(state):
            for (r2, c2) in enumerate(state):
                if (r1, c1) != (r2, c2):
                    num_conflicts += self.conflict(r1, c1, r2, c2)
        return 200 - num_conflicts

In [32]:
def find_neighbors(state, number_of_neighbors=70):
        
    neighbors = []
        
    for i in range(number_of_neighbors):
        new_state = problem.two_opt(state)
        neighbors.append(Node(new_state))
        state = new_state

    return neighbors


def hill_climbing(problem):
    
    iterations = 100

    current = Node(problem.initial)
    
    while iterations:
        neighbors = find_neighbors(current.state, 100)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) >= problem.value(current.state):
            current.state = neighbor.state
        iterations -= 1
        
    return current.state

def hill_climbing_random_restart(problem, restart_count = 1):
    
    current = Node(problem.initial)
    for i in range(restart_count):
        iterations = 70
    
        while iterations:
            neighbors = find_neighbors(current.state)
            if not neighbors:
                break
            neighbor = argmax_random_tie(neighbors,key=lambda node: problem.value(node.state))
            if problem.value(neighbor.state) >= problem.value(current.state):
                current.state = neighbor.state
            iterations -= 1
            
        # after the iterations, we check for a goal node and quit the random-restart.
        if(current.state == problem.goal):
            break

        # we shuffle the current state - essentially starting at a new random state.
        current.state = list(current.state)    
        random.shuffle(current.state) 
        current.state = tuple(current.state)
        
    return current.state

def exp_schedule(k=2.1, lam=0.01, limit=1000):
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)
    

def simulated_annealing(problem, schedule=exp_schedule()):
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = find_neighbors(current.state, 100)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice


In [33]:
number_of_problems = 50
initial_position = (1, 4, 2, 0, 7, 5, 3, 6, 8)
problems = []
for i in range(number_of_problems):
    random.shuffle(list(initial_position))
    initial_position = tuple(initial_position)
    e = EightPuzzle(initial_position)
    problems.append(e)

goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)

number_correct = 0
t0 = time.time()
for problem in problems:
    path = hill_climbing(problem)
    if(path == goal):
        number_correct += 1
t1 = time.time()

print("8 puzzle - hill-climbing")
print("number of problems: ", number_of_problems)
print(number_correct * 100 / number_of_problems, " percent correct")
print("time taken: ", t1 - t0)
print("")

number_correct = 0
t0 = time.time()
for problem in problems:
    path = hill_climbing_random_restart(problem, 10)
    if(path == goal):
        number_correct += 1
t1 = time.time()

print("8 puzzle - random-restart hill-climbing")
print("number of problems: ", number_of_problems)
print(number_correct * 100 / number_of_problems, " percent correct")
print("time taken: ", t1 - t0)
print("")

number_correct = 0
t0 = time.time()
for problem in problems:
    path = simulated_annealing(problem)
    if(path == goal):
        number_correct += 1
t1 = time.time()

print("8 puzzle - simulated annealing")
print("number of problems: ", number_of_problems)
print(number_correct * 100 / number_of_problems, " percent correct")
print("time taken: ", t1 - t0)
print("")

#########################################################################

problems = []
for i in range(number_of_problems):
    e = EightQueensProblem()
    problems.append(e)

number_correct = 0
t0 = time.time()
for problem in problems:
    path = hill_climbing(problem)
    if(problem.value(path) == 200):
        number_correct += 1
t1 = time.time()

print("8 queens - hill-climbing")
print("number of problems: ", number_of_problems)
print(number_correct * 100 / number_of_problems, " percent correct")
print("time taken: ", t1 - t0)
print("")

number_correct = 0
t0 = time.time()
for problem in problems:
    path = hill_climbing_random_restart(problem, 10)
    if(problem.value(path) == 200):
        number_correct += 1
t1 = time.time()

print("8 queens - random-restart hill-climbing")
print("number of problems: ", number_of_problems)
print(number_correct * 100 / number_of_problems, " percent correct")
print("time taken: ", t1 - t0)
print("")

number_correct = 0
t0 = time.time()
for problem in problems:
    path = simulated_annealing(problem)
    if(problem.value(path) == 200):
        number_correct += 1
t1 = time.time()

print("8 queens - simulated annealing")
print("number of problems: ", number_of_problems)
print(number_correct * 100 / number_of_problems, " percent correct")
print("time taken: ", t1 - t0)
print("")

8 puzzle - hill-climbing
number of problems:  5
80.0  percent correct
time taken:  0.5718932151794434

8 puzzle - random-restart hill-climbing
number of problems:  5
100.0  percent correct
time taken:  0.468447208404541

8 puzzle - simulated annealing
number of problems:  5
0.0  percent correct
time taken:  1.736462116241455

8 queens - hill-climbing
number of problems:  5
100.0  percent correct
time taken:  1.3798999786376953

8 queens - random-restart hill-climbing
number of problems:  5
0.0  percent correct
time taken:  6.945120096206665

8 queens - simulated annealing
number of problems:  5
100.0  percent correct
time taken:  2.122053623199463

